# Import Tools

In [17]:
import os
import supervision as sv
import cv2
from ultralytics import YOLO
import torch

HOME = os.getcwd()
HOME

'C:\\Users\\USER\\Downloads\\DeepFashion2\\DeepFashion2'

In [5]:
!pip install -q ultralytics supervision --upgrade supervision
from ultralytics import YOLO
import supervision as sv

In [6]:
DATA_PATH = f"{HOME}/yolov9/deepfashion2-m-11k-1"
IMAGE_PATH = f"{HOME}/yolov9/deepfashion2-m-11k-1/train/images"
WEIGHT_PATH = f"{HOME}/yolov9/weights/yolov9c.pt"
# SINGLE_IMAGE = IMAGE_PATH + "/000001_png.rf.3efc63de2fe2bab8882abc87437bf048.jpg"
SINGLE_IMAGE = "yolov9/mytest/5.jpeg"

# Get the model

In [7]:
model = YOLO(WEIGHT_PATH)
model.info()

YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

# Get the data

In [11]:
source_video_path = f"{HOME}/yolov9/myvid/background video _ people _ walking _.mp4"

# ByteTrack

In [19]:
COLOR_ANNOTATOR = sv.ColorAnnotator()
LABEL_ANNOTATOR = sv.LabelAnnotator(text_color=sv.Color.from_hex("#000000"))
tracker = sv.ByteTrack(minimum_matching_threshold=0.5)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
video_info = sv.VideoInfo.from_video_path(video_path=source_video_path)
frames_generator = sv.get_video_frames_generator(source_video_path)

frame = next(frames_generator)
resolution_wh = frame.shape[1], frame.shape[0]

for frame in frames_generator:
    results = model(frame, verbose=False, device=device, conf=0.3)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections.with_nms(threshold=0.7)
    detections = tracker.update_with_detections(detections)

    annotated_frame = frame.copy()

    annotated_frame = COLOR_ANNOTATOR.annotate(
        scene=annotated_frame,
        detections=detections,
    )

    labels = [
        f"#{tracker_id}"
        for tracker_id in detections.tracker_id
    ]

    annotated_frame = LABEL_ANNOTATOR.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels,
    )

    cv2.imshow("Processed Video", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()